In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

id_list = test_df['PassengerId']

train_df = train_df.drop(["PassengerId","Name", "Ticket", "Cabin"], axis=1);
test_df = test_df.drop(['PassengerId', 'Name', "Ticket", 'Cabin'], axis=1);

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
def feature_normalization(feature_vector):
    # scale to [-1, +1]
    max_value = max(feature_vector)
    min_value = min(feature_vector)
    mean_value = feature_vector.mean()
    return (feature_vector - mean_value) / (max_value - min_value)

def dummy(data, columns):
    for column in columns:
        if column not in data.columns:
            continue
        dummy_data = pd.get_dummies(data[column], drop_first=False)
        num = len(dummy_data.loc[0,:])
        #dummy_data.columns = [column+"_"+str(x+1) for x in range(num)]
        data = pd.concat([data, dummy_data], axis=1)
        data = data.drop(column, axis=1)
    return data


In [ ]:
train_df = dummy(train_df, ['Sex', "Embarked", 'Pclass'])
test_df = dummy(test_df, ['Sex', 'Embarked', 'Pclass'])
train_df.head()

In [ ]:
train_df.loc[train_df.Age.isnull()]

In [ ]:
def fare(data):
    data.loc[data.Fare.isnull(), 'Fare'] = data['Fare'].mean()
    data.Fare.astype(int)
    return data

def age(data):
    most = data.Age.mode()[0]
    data.loc[data.Age.isnull(), 'Age'] = most
    data.Age.astype(int)
    return data

test_df = fare(test_df)
train_df = age(train_df)
test_df = age(test_df)

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
train_df.head()

In [ ]:
model = LogisticRegression()
model.fit(train_df.drop('Survived', axis=1), train_df.Survived)
predictions = model.predict(test_df)

In [ ]:
data_d = {"PassengerId":id_list, "Survived":predictions}
output = pd.DataFrame(data_d)
output.to_csv("predictions.csv", index=False)